# Encapsulation

In [3]:
from typing import Dict, List, Optional
from datetime import datetime
import uuid

class InsufficientFundsError(Exception):
    """Custom exception for insufficient funds"""
    pass

class Transaction:
    """Class to represent individual transactions"""
    def __init__(self, amount: float, trans_type: str, description: str):
        self._id = str(uuid.uuid4())
        self._amount = amount
        self._timestamp = datetime.now()
        self._type = trans_type
        self._description = description

    @property
    def id(self) -> str:
        return self._id

    @property
    def details(self) -> Dict[str, str]:
        return {
            "id": self._id,
            "amount": f"${self._amount:.2f}",
            "type": self._type,
            "description": self._description,
            "timestamp": self._timestamp.isoformat()
        }

In [4]:
class BankAccount:
    """Main class demonstrating encapsulation"""
    # Class-level constants
    _MIN_BALANCE = 0.0
    _INTEREST_RATE = 0.02  # 2% annual interest
    
    def __init__(self, account_holder: str, initial_balance: float = 0.0):
        # Private attributes
        self.__account_number = str(uuid.uuid4())[:8]  # First 8 chars of UUID
        self.__balance = 0.0
        self.__holder = account_holder
        self.__transactions: List[Transaction] = []
        self.__is_frozen = False
        self.__pin = "0000"  # Default PIN

        # Initial deposit using property setters
        if initial_balance > 0:
            self.deposit(initial_balance, "Initial deposit")

    # Property getters
    @property
    def balance(self) -> float:
        """Read-only access to balance"""
        return self.__balance

    @property
    def account_number(self) -> str:
        """Read-only access to account number"""
        return self.__account_number

    @property
    def holder(self) -> str:
        """Read-only access to account holder"""
        return self.__holder

    # Property for PIN with getter and setter
    @property
    def pin(self) -> str:
        """PIN getter - returns masked value"""
        return "****"

    @pin.setter
    def pin(self, new_pin: str):
        """PIN setter with validation"""
        if not self.__validate_pin(new_pin):
            raise ValueError("PIN must be exactly 4 digits")
        self.__pin = new_pin

    # Private helper methods
    def __validate_pin(self, pin: str) -> bool:
        return len(pin) == 4 and pin.isdigit()

    def __check_pin(self, pin: str) -> bool:
        return self.__pin == pin and not self.__is_frozen

    def __add_transaction(self, amount: float, trans_type: str, description: str):
        transaction = Transaction(amount, trans_type, description)
        self.__transactions.append(transaction)
        return transaction.id

    # Public methods with access control
    def deposit(self, amount: float, description: str = "Deposit") -> str:
        """Deposit money into the account"""
        if self.__is_frozen:
            raise PermissionError("Account is frozen")
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        
        self.__balance += amount
        return self.__add_transaction(amount, "DEPOSIT", description)

    def withdraw(self, amount: float, pin: str, description: str = "Withdrawal") -> str:
        """Withdraw money with PIN verification"""
        if not self.__check_pin(pin):
            raise PermissionError("Invalid PIN or account frozen")
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        if self.__balance - amount < self._MIN_BALANCE:
            raise InsufficientFundsError(f"Insufficient funds. Current balance: ${self.__balance:.2f}")
        
        self.__balance -= amount
        return self.__add_transaction(-amount, "WITHDRAWAL", description)

    def apply_interest(self, pin: str):
        """Apply monthly interest with PIN verification"""
        if not self.__check_pin(pin):
            raise PermissionError("Invalid PIN or account frozen")
        if self.__balance <= 0:
            return
        
        interest = self.__balance * (self._INTEREST_RATE / 12)  # Monthly interest
        self.__balance += interest
        self.__add_transaction(interest, "INTEREST", "Monthly interest payment")

    def get_transaction_history(self, pin: str) -> List[Dict[str, str]]:
        """Get transaction history with PIN verification"""
        if not self.__check_pin(pin):
            raise PermissionError("Invalid PIN or account frozen")
        return [t.details for t in self.__transactions]

    def freeze_account(self, pin: str):
        """Freeze account with PIN verification"""
        if not self.__check_pin(pin):
            raise PermissionError("Invalid PIN")
        self.__is_frozen = True

    def unfreeze_account(self, pin: str):
        """Unfreeze account with PIN verification"""
        if not self.__check_pin(pin):
            raise PermissionError("Invalid PIN")
        self.__is_frozen = False

In [5]:
# Demonstration
def main():
    # Create a bank account
    account = BankAccount("John Doe", 1000.0)
    
    try:
        # Basic operations
        print(f"Account Number: {account.account_number}")
        print(f"Holder: {account.holder}")
        print(f"Initial Balance: ${account.balance:.2f}")

        # Change PIN
        account.pin = "1234"
        
        # Perform transactions
        deposit_id = account.deposit(500.0, "Salary")
        print(f"Deposit transaction ID: {deposit_id}")
        print(f"New Balance: ${account.balance:.2f}")

        withdraw_id = account.withdraw(200.0, "1234", "Shopping")
        print(f"Withdrawal transaction ID: {withdraw_id}")
        print(f"New Balance: ${account.balance:.2f}")

        # Apply interest
        account.apply_interest("1234")
        print(f"Balance after interest: ${account.balance:.2f}")

        # View transaction history
        print("\nTransaction History:")
        for trans in account.get_transaction_history("1234"):
            print(f"  {trans['type']}: {trans['amount']} - {trans['description']} "
                  f"({trans['timestamp']})")

        # Demonstrate protection
        # This will fail: account.__balance = 1000000  # AttributeError
        # This will fail: account.withdraw(500, "9999")  # PermissionError
        
        # Freeze account
        account.freeze_account("1234")
        try:
            account.withdraw(100, "1234")
        except PermissionError as e:
            print(f"\nFailed withdrawal: {e}")

    except (ValueError, InsufficientFundsError, PermissionError) as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

Account Number: c3634ca4
Holder: John Doe
Initial Balance: $1000.00
Deposit transaction ID: 44de984e-f92e-441d-b7c5-f95811f299a4
New Balance: $1500.00
Withdrawal transaction ID: d5907eaa-5ed3-467e-88b8-ce4d968ed869
New Balance: $1300.00
Balance after interest: $1302.17

Transaction History:
  DEPOSIT: $1000.00 - Initial deposit (2025-03-01T16:40:23.269963)
  DEPOSIT: $500.00 - Salary (2025-03-01T16:40:23.270654)
  WITHDRAWAL: $-200.00 - Shopping (2025-03-01T16:40:23.270732)
  INTEREST: $2.17 - Monthly interest payment (2025-03-01T16:40:23.270771)

Failed withdrawal: Invalid PIN or account frozen
